In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/sqsq.csv")
df = df.drop(df[df.Label == 'Label'].index)

NORM_LEN = 2000
DDOS_LEN = 2000
BRUTE_LEN = 1500
SQL_LEN = 4500
data1 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/norm.csv")
data2 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/ddos.csv")
data3 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/nb1.csv")
data1 = data1.sample(NORM_LEN)
data2 = data2.sample(DDOS_LEN)
data3 = data3.sample(BRUTE_LEN)
df = df.sample(SQL_LEN)
data = pd.concat([data1, data2, data3, df], ignore_index=True, sort=False)
shuffle = data.sample(frac=1, random_state=101).reset_index()

<ipython-input-21-d0e3ed57e9db>:4: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/sqsq.csv")


In [22]:
shuffle.fillna(0, inplace=True)
df = shuffle

In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [24]:
test_df = df.copy()

In [25]:
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.dropna(how="all", inplace=True)

number_inf = test_df[test_df == np.inf].count()

In [26]:
def ip2int(ip):
    if ip == 0:
      ip = '0.0.0.0'
    ip_list = list(map(int, ip.split('.')))
    num = 0
    for i in ip_list:
        num *= 256
        num += i
    return num

In [27]:
def delete_first_zero(val):
    if val[0] == '0':
        return val[1:]
    return val

def time2int(tm):
    tm_split = tm.split(' ')
    day_info = tm_split[0].split('/')
    day_info = list(map(delete_first_zero, day_info))
    day_info = list(map(int, day_info))
    day_amount = day_info[0] + (day_info[1] - 1) * 30 + (day_info[2] - 1970) * 365
    
    time_info = tm_split[1].split(':')
    time_info = list(map(delete_first_zero, time_info))
    time_info = list(map(int, time_info))
    sec_amount = 60 * (time_info[0] * 60 + time_info[1]) + time_info[2]
    
    if len(tm_split) == 3 and tm_split[-1] == 'PM':
        sec_amount += 3600 * 12
    sec_amount += day_amount * 24 * 3600
    
    return sec_amount

In [28]:
y = test_df['Label']
test_df = test_df.drop(['Label'], axis=1)
test_df['Src IP'] = test_df['Src IP'].apply(ip2int)
test_df['Dst IP'] = test_df['Dst IP'].apply(ip2int)
test_df['Timestamp'] = test_df['Timestamp'].apply(time2int)
test_df.head()

,index,Unnamed: 0,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,6676,0.0,0,0,0.0,0,443,6,1519005242,60867610,15,15,1050,5626,686,0,70.0,182.600657,1460,0,375.066667,591.704374,109.680666,0.492873,2098883.103,4171313.846,10200000,5,60900000,4347686.429,5199639.806,10200000,231,60900000,4346861.571,5202088.774,10200000,10,0,0,0,0,312,384,0.246436,0.246436,0,1460,215.354839,451.453766,203810.5032,0,0,1,1,0,0,0,1,1,222.533333,70.0,375.066667,0,0,0,0,0,0,15,1050,15,5626,8192,115,9,20,16651.16667,12876.40941,42935,11365,10100000.0,47164.77477,10200000,10100000
1,6421,0.0,0,0,0.0,0,4321,6,1519036813,2,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,1000000.0,2.0,0.0,2,2,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,24,20,500000.0,500000.0,0,0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,1,0.0,0.0,0.0,0,0,0,0,0,0,1,0,1,0,1024,0,0,24,0.0,0.0,0,0,0.0,0.0,0,0
2,9834,0.0,0,0,0.0,0,22,6,1519008137,111,2,0,0,0,0,0,0,0,0,0,0,0,0,18018.01802,111,0,111,111,111,111,0,111,111,0,0,0,0,0,0,0,0,0,40,0,18018.01802,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,279,-1,0,20,0,0,0,0,0,0,0,0
3,8492,0.0,0,0,0.0,0,53,17,1519013635,12353,1,1,40,101,40,40,40,0,101,101,101,0,11414.23136,161.9039909,12353,0,12353,12353,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,80.95199547,80.95199547,40,101,60.33333333,35.21836642,1240.333333,0,0,0,0,0,0,0,0,1,90.5,40,101,0,0,0,0,0,0,1,40,1,101,-1,-1,0,8,0,0,0,0,0,0,0,0
4,9982,0.0,0,0,0.0,0,53,17,1519043215,879,1,1,40,139,40,40,40.0,0.0,139,139,139.0,0.0,203640.5006,2275.312856,879.0,0.0,879,879,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,8,8,1137.656428,1137.656428,40,139,73.0,57.157677,3267.0,0,0,0,0,0,0,0,0,1,109.5,40.0,139.0,0,0,0,0,0,0,1,40,1,139,-1,-1,0,8,0.0,0.0,0,0,0.0,0.0,0,0


In [29]:
IMPORTANT_COLS = ['Src IP',
                  'Dst IP',
                  'Timestamp',
                  'Src Port',
                  'Dst Port',
                  'Protocol','Flow Duration',
                  'Flow Byts/s',
                  'Flow Pkts/s',
                  'Pkt Len Var',
                  'Down/Up Ratio'
                 ]

In [30]:
test_df = test_df.drop([i for i in list(test_df.columns) if i not in IMPORTANT_COLS], axis=1)
test_df.head()

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Flow Byts/s,Flow Pkts/s,Pkt Len Var,Down/Up Ratio
0,0,0.0,0,443,6,1519005242,60867610,109.680666,0.492873,203810.5032,1
1,0,0.0,0,4321,6,1519036813,2,0.0,1000000.0,0.0,1
2,0,0.0,0,22,6,1519008137,111,0,18018.01802,0,0
3,0,0.0,0,53,17,1519013635,12353,11414.23136,161.9039909,1240.333333,1
4,0,0.0,0,53,17,1519043215,879,203640.5006,2275.312856,3267.0,1


In [31]:
test_df = test_df.fillna(0)

In [32]:
test_df = test_df.astype(float)

In [33]:
df_z_scaled = test_df.copy()
  
# apply normalization techniques
for column in df_z_scaled.columns:
    df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / df_z_scaled[column].std()
df_z_scaled = df_z_scaled.fillna(0)
df_z_scaled.head()

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Flow Byts/s,Flow Pkts/s,Pkt Len Var,Down/Up Ratio
0,-0.740232,-0.614641,-0.733279,-0.343581,-0.495681,0.458283,1.585483,0.0,0.0,1.098342,0.377469
1,-0.740232,-0.614641,-0.733279,-0.089372,-0.495681,0.458631,-0.406621,0.0,0.0,-0.361261,0.377469
2,-0.740232,-0.614641,-0.733279,-0.371178,-0.495681,0.458314,-0.406618,0.0,0.0,-0.361261,-1.166371
3,-0.740232,-0.614641,-0.733279,-0.369146,1.875688,0.458375,-0.406217,0.0,0.0,-0.352378,0.377469
4,-0.740232,-0.614641,-0.733279,-0.369146,1.875688,0.458701,-0.406593,0.0,0.0,-0.337864,0.377469


In [34]:
y = y.str.replace('ddos', '1').str.replace('Benign','0').str.replace('FTP-BruteForce','2').str.replace('Infilteration','3')
y = y.astype(np.int64)

In [35]:
y.head()

0    0
1    3
2    0
3    0
4    0
Name: Label, dtype: int64

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_z_scaled, y, test_size=0.3, random_state=42)

In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
params = {
    'multi_class' : ['auto', 'ovr', 'multinomial'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'penalty':['None', 'elasticnet', 'l1', 'l2'],
    'C':[0.001, 0.01, 0.1, 1, 10, 100]
}
params = {
    'multi_class' : ['auto'],
    'solver': ['liblinear'],
    'penalty':['l1'],
    'C':[100]
}
gridsearch = GridSearchCV(lg, params,return_train_score=True, refit=True)
gridsearch.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [100], 'multi_class': ['auto'], 'penalty': ['l1'],
                         'solver': ['liblinear']},
             return_train_score=True)

In [38]:
gridsearch.best_params_

{'C': 100, 'multi_class': 'auto', 'penalty': 'l1', 'solver': 'liblinear'}

In [39]:
y_pred = gridsearch.predict(X_test)

In [43]:
from sklearn.metrics import classification_report, accuracy_score
print('Results on the test set:')
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.81      0.97      0.89      1548
           1       0.99      1.00      1.00       569
           2       0.98      1.00      0.99       476
           3       0.61      0.14      0.23       407

    accuracy                           0.87      3000
   macro avg       0.85      0.78      0.78      3000
weighted avg       0.85      0.87      0.84      3000

0.869
